# Generating Speech

## "Get me a flight to JFK"

In [48]:
from gtts import gTTS
import os

mytext = 'Good Morning. I need to fly to New York and say over in London.\
           Get me a flight to JFK and a hotel in central London'

language = 'en'

tts = gTTS(text=mytext, lang=language) 

tts.save('flight.mp3')
os.system("mpg321 flight.mp3")

0

In [53]:
# convert to wav file 
from pydub import AudioSegment
from pydub.playback import play

# files                                                                         
src = "flight.mp3"
dst = "flight.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)

sound.export(dst, format="wav")
play(sound)


# Transcribing Speech

In [56]:
import speech_recognition as sr

r = sr.Recognizer()
# use the audio file as the audio source
r = sr.Recognizer()

with sr.WavFile('flight.wav') as source:
     audio = r.record(source, duration=10)  # read the entire audio file 
     print("Transcription: " + r.recognize_google(audio, language='en-US'))

Transcription: London


# Understanding Grammar

In [69]:
import nltk
from nltk import pos_tag, word_tokenize

sentence = "Get me a flight to JFK"

tokens = word_tokenize(sentence)
tagged_tokens = pos_tag(tokens)
print(tagged_tokens)


[('Get', 'VB'), ('me', 'PRP'), ('a', 'DT'), ('flight', 'NN'), ('to', 'TO'), ('JFK', 'VB')]
